In [36]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import base64
import imagehash
import mysql.connector
import os
import re

In [37]:
# GENERAL CONFIGS

ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_2"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\dnmarchives\sheep\sheep\2013-12-03"

In [38]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

In [39]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_2 ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  price VARCHAR(50),"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  has_image VARCHAR(25) NOT NULL DEFAULT 'has_image',"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [40]:
def update_image(product_id, value):
    query = "UPDATE products_tmp_2 SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [41]:
def get_content(content):
    clear_content = ""

    if content is not None:
        raw_c = content.text

        clear_content = re.sub(r'[\t\r\n]', ' ', raw_c)
        clear_content = re.sub(' +',' ', clear_content).lstrip()
        
    return clear_content

In [44]:
original_images = []

folder = Path(f"{MAIN_FOLDER}\market\item")

for file in os.listdir(folder):
    page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            
    if page_content is not None:
        ## GET DESCRIPTION
        raw_description = page_content.find("td", attrs={"style": "background-color: #F8F8F8;"})
        description = get_content(raw_description)

        ## GET TITLE
        raw_title = page_content.find("h4")
        title = get_content(raw_title)

        ## GET SELLER
        raw_seller = page_content.find("a", attrs={"title": "Send a message"})
        seller = get_content(raw_seller)

        if description != "" or title != "":
            # Insert product into database
            query = "INSERT INTO products_tmp_2 (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
            values = (title, description, "Sheep", seller_name, "t")

            cursor.execute(query, values)
            conn.commit()

            product_id = cursor.lastrowid

            ## GET IMAGE
            raw_image = page_content.find("img")
            
            if raw_image is not None:
                try:
                    image = raw_image["src"]
                except IndexError:
                    print(f"Image {product_id} has no src")
                    update_image(product_id, "no_image")
                
                if image is not "":
                    try:
                        data = "" + image.replace("data:image/jpeg;base64,", "")
                        imgdata = base64.b64decode(data)

                        img_filename = Path(f"D:\images\p\{product_id}.jpg")

                        with open(img_filename, 'wb') as f:
                            f.write(imgdata)
                            f.close
                    except:
                        print(f"Image {product_id} has error. Except")
                        update_image(product_id, "no_image")
            else:
                    print(f"Image {product_id} has no image")
                    update_image(product_id, "no_image")

Image 196 has error. Except
Image 265 has error. Except
Image 471 has error. Except
Image 1830 has error. Except
Image 2184 has error. Except
Image 2923 has error. Except
Image 3005 has error. Except
Image 3175 has error. Except
Image 3516 has error. Except
Image 3612 has error. Except
Image 3730 has error. Except
Image 3899 has error. Except
Image 4177 has error. Except
Image 4285 has error. Except
Image 4340 has error. Except
Image 4787 has error. Except
Image 4803 has error. Except
Image 4871 has error. Except
Image 4873 has error. Except
Image 4968 has error. Except
Image 4988 has error. Except
